In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

In [3]:
os.getcwd()

'/app'

In [4]:
os.listdir()

['.ipynb_checkpoints', 'graphs', 'main.ipynb', 'requirements.txt']

In [5]:
from graphs.testing import test

In [8]:
test()

Hello World


In [2]:
from qdrant_client import QdrantClient

qdrant = QdrantClient("http://qdrant:6333")

In [ ]:
qdrant.info()

VersionInfo(title='qdrant - vector search engine', version='1.14.1', commit='530430fac2a3ca872504f276d2c91a5c91f43fa0')